# MovieRecommender

### A ai-based movie recommendation system trained with MovieLens dataset


## The dataset - a brief description

To train a neural network, which should be able to recommend you good movies based on your rating-history, a large-scale dataset is needed. Therefore the [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) dataset is used. It contains:
- 100.000 Ratings, each rating contains a one-to-five-star rating made by one user (with userID XY) on one movie (with userID YZ).
- 943 Users, labeled with an userID, age, gender, occupation and zip-code
- 1682 Movies, labeled with an movieID, title, release dates, IMDb URL and a list of genres

You can find the full documentation of the dataset structure under ML_100_INFO.md


### Acknowledgement

The dataset was published here:
F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets:
History and Context. ACM Transactions on Interactive Intelligent
Systems (TiiS) 5, 4, Article 19 (December 2015), 19 pages.
DOI=http://dx.doi.org/10.1145/2827872


### Importing data

First the data musst be imported in a usable dataformat to do some pre-processing. Also some analyzation can be done to get a feeling for the dataset, its content, diversity and limitations.
To access the data its useful to define some classes: (LEARNING: Better just use pandas_df...)

In [44]:
import DataClasses

Then its time to start importing the dataset (and required modules)

In [275]:
#Module Imports
import pandas as pd
import csv
from pathlib import Path
import numpy as np
#Dataset import
IMPORT_ONLINE = False


data_dir = Path("ml-100k")

ratingFile = data_dir / "u.data"
df_ratings = pd.read_csv(ratingFile, delimiter='\t', encoding="latin",names=["userId","movieId","rating","t_stamp"],index_col=False)
movieFile = data_dir / "u.item"
# print(df_ratings)
df_movies = pd.read_csv(movieFile, delimiter='|', encoding="latin",names=["id","title","date","v_date","url","unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"])
# print(df_movies)
print(f"Sanity check:\nThe movie {df_movies['title'][212]} contains the genres: {[df_movies.columns[i] for i in range(len(df_movies.columns)) if df_movies.iloc[212][df_movies.columns[i]] == 1]}")
userFile = data_dir / "u.user"
df_users = pd.read_csv(userFile, delimiter='|', encoding="latin",names=["userId","age","gender","occupation","zip"])
# print(df_users)
print(f"Sanity check:\nThe user 111 ({df_users['gender'][110]} {df_users['age'][110]}) works as {df_users['occupation'][110]} - ID: {df_users['userId'][110]}")

Sanity check:
The movie Room with a View, A (1986) contains the genres: ['Drama', 'Romance']
Sanity check:
The user 111 (M 57) works as engineer - ID: 111


In [ ]:

# DataClasses.importAsDataClasses() #not used bd too complicated
if(IMPORT_ONLINE):
    print()
    #TODO: Import from url
else:
    with open(r'ml-100k\u.data') as ml_100_data: #ml-100k\u.data
        data_csv = csv.reader(ml_100_data, delimiter ='\t')
        ratings = list(data_csv)

    movies = list()
    with open(r'ml-100k\u.item') as movies_data: #ml-100k\u.data
        movies_csv = csv.reader(movies_data, delimiter ='|')
        data_dir = Path("ml-100k")
        itemFile = data_dir / "u.item"
        df_movies = pd.read_csv(itemFile)
        for row in movies_csv:
            movies.append(Movie(id=row[0], title=row[1], r_date=row[2], v_date=row[3], url=str(row[4]), g_list=row[5:24]))
    print(f"Sanity check:\nThe movie {movies[0].title} contains the genres: {movies[0].get_genres()}")

    users = list()
    with open(r'ml-100k\u.user') as users_data: #ml-100k\u.data
        users_csv = csv.reader(users_data, delimiter ='|')
        for row in users_csv:
            users.append(User(id=row[0],age=row[1],gender=row[2],occ=row[3],zip=row[4]))
    print(f"Sanity check:\nThe user 111 ({users[110].gender} {users[110].age}) works as {users[110].occupation} - ID: {users[110].id}")

### Analyzation


In [ ]:
# Analyse: Best Movie, average ratings, average rating counts,
# #TODO: plot ratings, best movie and stuff

### Preprocessing
After successfully importing all relevant data, the next step is to pre-process the data so it fits our nn-input

## The nn-architecture
### Goal
The goal of the recommender system is to predict the best movie recommendation for user with respect to his ratings on other movies and the movie-ratings of other users in general. The easiest way to determine the "best" movie is to predict the users rating on unwatched movies and pick the highest rated one.
### First approach: Genre-recommendation
The first idea to predict the users rating (before diving into collaborative filtering) was to calculate every users preferences based on his rating-history and then train classical dense neural-network with it:

``Picture``

First the genre-preference calculation has to be done. The output of it should be a weighted genre list containg values in \[0,1]
The input of the nn is defined by the count of genres: 18 movie genres + 18 preference-weighted genres

In [ ]:
#approach specific pre-processing
ratings.sort() #by time and user
#change movie id to genre or include at row[4:23]

In [297]:
#User genre preference calculation based on all other ratings in the training set
def calc_pref(userID, movieID):
    #get all ratings:
    user_ratings = df_ratings.loc[(df_ratings["userId"] == userID) & (df_ratings["movieId"] != movieID)]
    user_movies = df_movies.loc[df_movies["id"].isin(user_ratings["movieId"])]

    # add rating
    user_movies["rating"] = [int(user_ratings.loc[(user_ratings["movieId"]==id),"rating"]) for id  in user_movies["id"]]
    genre_table = user_movies.loc[:,"unknown":"rating"]# #only  (and rating) of genres
    num_movies = len(user_movies)
    if num_movies:
        g_mat, r_mat = np.hsplit(genre_table.to_numpy(), [-1]) # seperating rating column from rest
        r_mat = np.tile(r_mat, (1,19))
        g_mat = g_mat * r_mat / 5 #weight g_mat with rating
        pref_mat = np.asarray([np.sum(g_mat[:,i]) for i in range(np.shape(g_mat)[1])])
        pref_mat = pref_mat / num_movies #normalize to num_movies
        # multiply with rating
        return pref_mat
    else:
        return 0

res = calc_pref(12,23)
print(f"Check: User 50 has preference for genres:\n{calc_pref(50,101)}")

Check: User 50 has preference for genres:
[0.         0.         0.         0.         0.         0.075
 0.13333333 0.04166667 0.45833333 0.         0.025      0.06666667
 0.04166667 0.075      0.225      0.         0.08333333 0.01666667
 0.        ]


C:\Users\manue\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\manue\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:

for user in users:
    pref_genres = []
    for user_rating in ratings:
        if user_rating[0] == user.id: #only ratings of user
            genre = [movie.genre_list for movie in movies if movie.id == user_rating[1]][:] #find genres of rated movie
            print(genre)
            pref_genres.append(calc_pref(pref_genres, genre, user_rating[2])) #ad prefered weights to list
        #add movie_genres and pref_gen to row


# Findings and outlook